In [1]:
import numpy as np
from mindspore import context
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceFieldBase
from mindsponge import DynamicUpdater

from mindsponge.potential import BondEnergy, AngleEnergy
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.function import VelocityGenerator
from mindsponge.control import LeapFrog, BerendsenThermostat

## 图模式选择与GPU选择

一般情况下，使用MindSpore的静态图模式GRAPH_MODE会取得更好的速度增益，而动态图模式PYNATIVE_MODE更多的被用于进行Debug。在硬件平台上，如果没有华为的Ascend专用硬件，一般都是在GPU上运行，才有一定的速度保障。如果有多张卡，可以通过device_id进行选择。

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 创建分子对象

一个分子，可以是三个原子组成的水分子，也可以是上万个原子组成的蛋白质。在MindSponge中，为不同类型的分子提供了多种定义形式。当定义一个小体系$H_2O$时，我们可以直接用常规的列表来定义分子中所包含的原子种类、原子坐标以及键连信息等。在后续的案例中我们也会讲到，还可以通过配置文件等方式来进行定义，这两种方式都是支持的。

In [3]:
system = Molecule(atoms=['O', 'H', 'H'],
                  coordinate=[[0, 0, 0], [0.1, 0, 0], [-0.0333, 0.0943, 0]],
                  bond=[[[0, 1], [0, 2]]])

## 能量项定义

不同的分子体系会有不同的能量项的需求。比如当前定义的$H_2O$分子，因为只有3个原子，因此涉及到的能量项只有近程相互作用中的键相互作用和角相互作用：

$$
E_{bond}=\frac{1}{2}k_b(l-l_0)^2\\
E_{angle}=\frac{1}{2}k_a(a-a_0)^2
$$

In [4]:
bond_energy = BondEnergy(index=system.bond,
                         force_constant=[[345000, 345000]],
                         bond_length=[[0.1, 0.1]])

In [5]:
angle_energy = AngleEnergy(index=[[1, 0, 2]],
                           force_constant=[[383]],
                           bond_angle=[[109.47 / 180 * np.pi]])

In [6]:
potential = ForceFieldBase(energy=[bond_energy, angle_energy])

## 速度生成器

根据指定的温度，我们可以计算出系统平均运动动能，进而通过标准正态分布生成随机的一组原子运动速度，用于初始化系统。

In [7]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)

TotalTime = 0.157032, [19]
[parse]: 0.0221374
[symbol_resolve]: 0.0702502, [1]
    [Cycle 1]: 0.0697669, [1]
        [resolve]: 0.0697553
[combine_like_graphs]: 2.66545e-06
[inference_opt_prepare]: 0.000401873
[abstract_specialize]: 0.0494782
[auto_monad]: 0.000281934
[inline]: 6.17094e-06
[py_pre_ad]: 1.46404e-06
[pynative_shard]: 3.58559e-06
[pipeline_split]: 4.88199e-06
[optimize]: 0.00598948, [15]
    [simplify_data_structures]: 0.00013376
    [opt_a]: 0.00518617, [2]
        [Cycle 1]: 0.00425204, [25]
            [expand_dump_flag]: 5.26011e-06
            [switch_simplify]: 0.00027016
            [a_1]: 0.00186183
            [recompute_prepare]: 7.12462e-06
            [updatestate_depend_eliminate]: 1.08667e-05
            [updatestate_assign_eliminate]: 8.59797e-06
            [updatestate_loads_eliminate]: 8.48807e-06
            [parameter_eliminate]: 3.05101e-06
            [a_2]: 0.000274925
            [accelerated_algorithm]: 6.96629e-06
            [auto_parallel]: 4.6

## 动力学积分器

如果不对系统施加任何的限制，任由系统去演化的话，就只需要配置integrator积分器，常见的算法有LeapFrog与VelocityVerlet，在MindSponge的control中都是支持的。通过前面配置的各项势能项，利用MindSpore框架的自动微分，就可以计算出当前受力以及下一步的位移，进而实现分子系统的演化。

如果需要对系统施加以一定的控制，比如控制系统的温度，或者是控制系统的压强，在MindSponge的control中可以调用BerendsenThermostat、Barostat等算法，来跑一个NVT的系统，避免系统演化中可能出现的温度震荡现象。

后续在MindSponge的新版本中，还会对LINCS、SETTLE等键长约束算法进行支持，可以在动力学演化的过程中确保不会出现不合理的键长拉伸，最大程度上的保障分子系统键连关系的合理性。

In [8]:
opt = DynamicUpdater(system,
                     integrator=LeapFrog(system),
                     thermostat=BerendsenThermostat(system, 300),
                     time_step=1e-3,
                     velocity=velocity)

In [9]:
md = Sponge(system, potential, opt)

TotalTime = 0.339354, [19]
[parse]: 0.00587304
[symbol_resolve]: 0.057146, [1]
    [Cycle 1]: 0.056314, [1]
        [resolve]: 0.0563009
[combine_like_graphs]: 1.44728e-06
[inference_opt_prepare]: 0.000434842
[abstract_specialize]: 0.241059
[auto_monad]: 0.000615519
[inline]: 2.15881e-06
[py_pre_ad]: 1.62609e-06
[pynative_shard]: 1.58511e-06
[pipeline_split]: 5.76861e-06
[optimize]: 0.025328, [15]
    [simplify_data_structures]: 0.000273107
    [opt_a]: 0.024218, [2]
        [Cycle 1]: 0.0232167, [25]
            [expand_dump_flag]: 1.09021e-05
            [switch_simplify]: 0.000576558
            [a_1]: 0.00530642
            [recompute_prepare]: 2.53115e-05
            [updatestate_depend_eliminate]: 1.73561e-05
            [updatestate_assign_eliminate]: 1.51228e-05
            [updatestate_loads_eliminate]: 1.53296e-05
            [parameter_eliminate]: 2.25008e-06
            [a_2]: 0.000652859
            [accelerated_algorithm]: 1.63987e-05
            [auto_parallel]: 3.33972e

## 信息回调

基于MindSpore的信息回调系统CallBack，我们可以创建一些符合自己需求的信息回调机制，比如使用RunInfo在屏幕上对系统能量进行输出，或者是通过WriteH5MD将轨迹输出到hdf5/h5md文件中。

关于h5md格式的轨迹输出，可以使用改进版的VMD-h5mdplugin进行可视化，相关安装和说明链接为：https://gitee.com/helloyesterday/VMD-h5mdplugin

In [10]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b01.h5md', save_freq=10, write_velocity=True, write_force=True)

In [11]:
md.run(1000, callbacks=[run_info, cb_h5md])

TotalTime = 0.0163505, [19]
[parse]: 0.00723182
[symbol_resolve]: 0.00106402, [1]
    [Cycle 1]: 0.001034, [1]
        [resolve]: 0.00102836
[combine_like_graphs]: 6.94767e-07
[inference_opt_prepare]: 2.78e-05
[abstract_specialize]: 0.000662819
[auto_monad]: 0.000122383
[inline]: 1.62236e-06
[py_pre_ad]: 1.30571e-06
[pynative_shard]: 1.16043e-06
[pipeline_split]: 2.00421e-06
[optimize]: 0.00188927, [15]
    [simplify_data_structures]: 8.42474e-06
    [opt_a]: 0.00152243, [2]
        [Cycle 1]: 0.000888854, [25]
            [expand_dump_flag]: 1.63168e-06
            [switch_simplify]: 9.95584e-06
            [a_1]: 0.0001512
            [recompute_prepare]: 3.65078e-06
            [updatestate_depend_eliminate]: 2.4857e-05
            [updatestate_assign_eliminate]: 4.692e-06
            [updatestate_loads_eliminate]: 4.00655e-06
            [parameter_eliminate]: 1.7751e-06
            [a_2]: 6.41923e-05
            [accelerated_algorithm]: 6.13928e-06
            [auto_parallel]: 1.8

上述运行结果的轨迹输出如下图所示，该结果展示使用了VMD-h5mdplugin插件，在VMD中对分子运行轨迹进行了可视化。

![](./docs/tutorial_b01.gif)